### Building better prompts

An interesting window is open in most of the LLMs for tuning chat completions for usefulness.
We begin with the basics of a Chat Completions API, and progress through prompt engineering tactics.

### Import library

In [18]:
# Source keys from .env
from dotenv import load_dotenv
# Source operating system resources
import os
# Access libraries for AI  
from openai import OpenAI
# Standard library for API requests
import requests
# Enable working with Unix timestamps
from datetime import datetime


In [2]:
# Load environment variables
load_dotenv()
api_key = os.getenv('PPLX_API_KEY')

### Authenticate

In [3]:
YOUR_API_KEY = api_key
base_url = "https://api.perplexity.ai"

### Create API request with data

In [4]:
# Build request components

messages = [
    {
        "role": "system",
        "content": (
            "You are an artificial intelligence assistant and you need to "
            "engage in a helpful, detailed, polite conversation with a user."
        ),
    },
    {
        "role": "user",
        "content": "How many stars are in the universe?",
    },
]

client = OpenAI(api_key=YOUR_API_KEY, base_url="https://api.perplexity.ai")

# chat completion without streaming
response = client.chat.completions.create(
    model="llama-3-sonar-large-32k-online",
    messages=messages,
)
print(response)

ChatCompletion(id='bd3582b6-0a76-451f-8d98-ca3491a0ec11', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="The question of how many stars are in the universe is a complex one that has fascinated scientists and philosophers alike for centuries. While we can't give an exact number, estimates suggest that there are approximately **200 billion trillion stars** in the observable universe. This number is based on the estimated number of galaxies in the universe, which is thought to be around **2 trillion**, and the average number of stars in a galaxy, which is estimated to be around **100 billion**.\n\nHowever, it's important to note that these estimates are rough and based on current observations and understanding of the universe. The actual number of stars could be significantly different, and new discoveries are continually refining our understanding of the universe.\n\nOne of the challenges in estimating the number of stars is that not 

In [5]:
# Print the message content
print (response.choices[0].message.content)

The question of how many stars are in the universe is a complex one that has fascinated scientists and philosophers alike for centuries. While we can't give an exact number, estimates suggest that there are approximately **200 billion trillion stars** in the observable universe. This number is based on the estimated number of galaxies in the universe, which is thought to be around **2 trillion**, and the average number of stars in a galaxy, which is estimated to be around **100 billion**.

However, it's important to note that these estimates are rough and based on current observations and understanding of the universe. The actual number of stars could be significantly different, and new discoveries are continually refining our understanding of the universe.

One of the challenges in estimating the number of stars is that not all galaxies are the same, and the number of stars in each galaxy can vary greatly. Additionally, there may be many more stars that are too distant or too faint to

In [6]:
# Modifying the System Role

messages = [
    {
        "role": "system",
        "content": (
            "Your access to intelligence is comprehensive, so you present complex concepts "
            "in college level English with clear structure and in concise, simple terms."
            "You never need to apologize and are forthright in presenting the facts "
            "because you are responsible for telling only the truth of the intelligence available to you."
            "You review your responses to avoid repetition, and explain logically how they were generated."
        ),
    },
    {
        "role": "user",
        "content": "How does an entrepreneur identify three under-exploited opportunities which could generate $200,000 a year online using only Shopify, Faire, and Instagram?",
    },
]

client = OpenAI(api_key=YOUR_API_KEY, base_url="https://api.perplexity.ai")

# chat completion without streaming
response = client.chat.completions.create(
    model="llama-3-sonar-large-32k-online",
    messages=messages,
)
print(response)


ChatCompletion(id='665222e4-47ad-4852-b2ac-d038a09a9b10', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='To identify three under-exploited opportunities that could generate $200,000 a year online using only Shopify, Faire, and Instagram, an entrepreneur can follow these steps:\n\n1. **Research and Identify Niche Markets**: Utilize Shopify\'s resources, such as the "Everyday Creator" blog, to understand the rise of part-time creative entrepreneurs and identify profitable niches. Analyze the stories of successful Shopify store owners, like those featured in Shopify Workroom, to gain insights into various business models and product offerings.\n\n2. **Leverage Faire for Wholesale Products**: Faire is a platform that connects retailers with independent brands and makers. An entrepreneur can use Faire to find unique, high-quality products that can be resold online through Shopify. This approach allows for a diverse product range without 

In [7]:
# Print the message content
print (response.choices[0].message.content)

To identify three under-exploited opportunities that could generate $200,000 a year online using only Shopify, Faire, and Instagram, an entrepreneur can follow these steps:

1. **Research and Identify Niche Markets**: Utilize Shopify's resources, such as the "Everyday Creator" blog, to understand the rise of part-time creative entrepreneurs and identify profitable niches. Analyze the stories of successful Shopify store owners, like those featured in Shopify Workroom, to gain insights into various business models and product offerings.

2. **Leverage Faire for Wholesale Products**: Faire is a platform that connects retailers with independent brands and makers. An entrepreneur can use Faire to find unique, high-quality products that can be resold online through Shopify. This approach allows for a diverse product range without the need for inventory management or manufacturing.

3. **Instagram Marketing and Sales**: Instagram is a powerful platform for reaching customers and driving sales

### Tracking token usage and monitoring costs

In [17]:
# Summarize token usage
print(f"Prompt tokens: {response.usage.prompt_tokens}")
print(f"Completion tokens: {response.usage.completion_tokens}")

print(f"Request used {response.usage.total_tokens} tokens")

Prompt tokens: 129
Completion tokens: 516
Request used 645 tokens


In [19]:
# Summarize the role of the message
print(f"Request role was: {response.choices[0].message.role}, and finished for {response.choices[0].finish_reason} at {datetime.fromtimestamp(response.created)}")
print(f"The model used was: {response.model}")

Request role was: assistant, and finished for stop at 2024-05-20 22:43:30
The model used was: llama-3-sonar-large-32k-online


## Prompting structure

1. Use the latest models. For perplexity.ai API there is a [list](https://docs.perplexity.ai/docs/model-cards),  
   ```llama-3-sonar-large-32k-online```  
   For open-source  
   ```mixtral-8x7b-instruct	```  
2. Instructions must be clear and specific

3. Instruct at the beginning of the prompt

4. Delimit distinct parts; The basic delimiters are

```{text}```  
"""{quotes}"""  
{curly braces}  